<h1 align="center">LAB1S1_p2. Preprocessing and Traditional
Language Representation using scikit-learn
tools</h1>

<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

### Put your names here

- Filip Polacik
- Kacper Multan

In [102]:
# IF YOU USE GOOGLE COLAB AND WANT READ A FILE
#from google.colab import drive
#drive.mount('/content/drive')

## Some help, if you need it

In [103]:
from nltk.book import texts
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import re

definition = {
    "a": "Bag of Words (binary)",
    "b": "Bag of Words (Term-Frequency) without normalization",
    "c": "Bigrams of Words (Term-Frequency) without normalization",
    "d": "Trigram of Characters (Term-Frequency) without normalization",
    "e": "TF-IDF based on words with 'l2' normalization",
    "f": "LSA based on TF-IDF of words (50 singular values)",
    "g": "LSA based on TF-IDF of words (100 singular values)"
}


# !pip install pandas
# !pip install nltk
# !pip install scikit-learn

## Load de dataset

In [104]:
filename = "EXIST2024_EN_examples.csv"

import csv

class CSVReader:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = []

    def read_csv(self):
        with open(self.file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file, delimiter='\t')
            for row in reader:
                self.data.append({
                    "id": int(row["id"]),
                    "text": row["text"],
                    "is_sexist": row["label"].strip().upper() == "YES",
                    "size": int(row["size"])
                })

    def get_data(self):
        return self.data

reader = CSVReader(filename)
reader.read_csv()

## Preprocess dataset

In [105]:
filename = "EXIST2024_EN_examples.csv"

import re

web_re = re.compile(r"https?:\/\/[^\s]+", re.U)

def preprocess(text):
    text = web_re.sub("", text)
    text = text.lower()
    return text

for tweet in reader.get_data():
    tweet["text"] = preprocess(tweet["text"])

In [106]:
from sklearn.feature_extraction.text import CountVectorizer

def compute_similarity(corpus, bow, sexist):
    cosine_sim = cosine_similarity(bow)
    maxi = 0
    max_ind = 0, 0

    for x in range(len(cosine_sim)):
        for y in range(len(cosine_sim[x])):
            if x != y:
                if cosine_sim[x][y] > maxi:
                    maxi = cosine_sim[x][y]
                    max_ind = x, y

    print(max_ind)
    tweet1, tweet2 = corpus[max_ind[0]], corpus[max_ind[1]]
    if sexist:
        print(f"Most similar sexist tweets:\n1: {tweet1}\n2: {tweet2}")
        print(f"Cosine Similarity: {cosine_sim[max_ind[0]][max_ind[1]]:.4f}")
    else:
        print(f"Most similar non sexist tweets:\n1: {tweet1}\n2: {tweet2}")
        print(f"Cosine Similarity: {cosine_sim[max_ind[0]][max_ind[1]]:.4f}")

### Binary

In [107]:


MyCorpus_sexist = [tweet["text"] for tweet in reader.get_data() if tweet["is_sexist"]]
MyCorpus_nonsexist = [tweet["text"] for tweet in reader.get_data() if not tweet["is_sexist"]]

vectorizer_bin = CountVectorizer(binary= True)
vectorizer_bin.fit(MyCorpus_sexist)
X_bag_of_words_bin = vectorizer_bin.transform(MyCorpus_sexist)
print("Vocabulary size for binary: ", len(vectorizer_bin.vocabulary_))

compute_similarity(MyCorpus_sexist, X_bag_of_words_bin, True)

print("--------------------------------")
vectorizer_bin = CountVectorizer(binary= True)
vectorizer_bin.fit(MyCorpus_nonsexist)
X_bag_of_words_bin = vectorizer_bin.transform(MyCorpus_nonsexist)
print("Vocabulary size for binary: ", len(vectorizer_bin.vocabulary_))

compute_similarity(MyCorpus_nonsexist, X_bag_of_words_bin, False)


Vocabulary size for binary:  6715
(1095, 1098)
Most similar sexist tweets:
1: @yayroger @victoriarossi @ionaguyf @metsdaddy2013 @byandrewwagner @themikebpeters dr. cox, does this shade of red make me look like a clown?no barbie. it makes you look like a prostitute who caters exclusively to clowns.
2: sharon: oh, loki, does this lipstick make me look like a clown?loki: no, barbie, no... it makes you look like a prostitute who caters exclusively *to* clowns.
Cosine Similarity: 0.7126
--------------------------------
Vocabulary size for binary:  10601
(872, 873)
Most similar non sexist tweets:
1: ♫ now playing : treat her like a lady (single version) by the temptations  
2: now playing:  the temptations - treat her like a lady   listen live: 
Cosine Similarity: 0.7628


### Bag of Words (Term-Frequency)

In [108]:
vectorizer_freq = CountVectorizer(binary= False)
vectorizer_freq.fit(MyCorpus_sexist)
X_bag_of_words_freq = vectorizer_freq.transform(MyCorpus_sexist)
print("Vocabulary size for frequency: ", len(vectorizer_freq.vocabulary_))

compute_similarity(MyCorpus_sexist, X_bag_of_words_freq, True)

print("--------------------------------")
vectorizer_freq = CountVectorizer(binary= False)
vectorizer_freq.fit(MyCorpus_nonsexist)
X_bag_of_words_freq = vectorizer_freq.transform(MyCorpus_nonsexist)
print("Vocabulary size for frequency: ", len(vectorizer_freq.vocabulary_))

compute_similarity(MyCorpus_nonsexist, X_bag_of_words_freq, False)

Vocabulary size for frequency:  6715
(1095, 1098)
Most similar sexist tweets:
1: @yayroger @victoriarossi @ionaguyf @metsdaddy2013 @byandrewwagner @themikebpeters dr. cox, does this shade of red make me look like a clown?no barbie. it makes you look like a prostitute who caters exclusively to clowns.
2: sharon: oh, loki, does this lipstick make me look like a clown?loki: no, barbie, no... it makes you look like a prostitute who caters exclusively *to* clowns.
Cosine Similarity: 0.7247
--------------------------------
Vocabulary size for frequency:  10601
(635, 639)
Most similar non sexist tweets:
1: @bleedthisway replay free woman breebylon &gt;&gt;&gt; flop this way
2: replay&gt;alice&gt;babylon&gt;free woman 
Cosine Similarity: 0.7778


### Bigram of Words (Term-Frequency)

In [109]:
bigram_vectorizer = CountVectorizer(analyzer='word',
                        ngram_range=(2,2),binary=False, stop_words="english")

counts = bigram_vectorizer.fit_transform(MyCorpus_sexist)
print("Vocabulary size for sexist bigrams of words: ", len(bigram_vectorizer.vocabulary_))
compute_similarity(MyCorpus_sexist, counts, True)

print("--------------------------------")
counts = bigram_vectorizer.fit_transform(MyCorpus_nonsexist)
print("Vocabulary size for non sexist bigrams of words: ", len(bigram_vectorizer.vocabulary_))
compute_similarity(MyCorpus_nonsexist, counts, False)



Vocabulary size for sexist bigrams of words:  13970
(1093, 1102)
Most similar sexist tweets:
1: "this boy you look like a prostitute" 
2: pov you look like a prostitute and have to go to a quince
Cosine Similarity: 0.5774
--------------------------------
Vocabulary size for non sexist bigrams of words:  22424
(1253, 1263)
Most similar non sexist tweets:
1: @jennfrey must be based on the bem sex role inventory and
2: bem sex role inventory is a problematic tool
Cosine Similarity: 0.6000


### Trigram

In [110]:
trigram_vectorizer = CountVectorizer(analyzer='word',
                        ngram_range=(3,3),binary=False, stop_words="english")

counts = trigram_vectorizer.fit_transform(MyCorpus_sexist)
print("Vocabulary size for trigrams of words: ", len(trigram_vectorizer.vocabulary_))
compute_similarity(MyCorpus_sexist, counts, True)

print("--------------------------------")
counts = trigram_vectorizer.fit_transform(MyCorpus_nonsexist)
print("Vocabulary size for non sexist trigrams of words: ", len(trigram_vectorizer.vocabulary_))
compute_similarity(MyCorpus_nonsexist, counts, False)

Vocabulary size for trigrams of words:  13947
(1095, 1098)
Most similar sexist tweets:
1: @yayroger @victoriarossi @ionaguyf @metsdaddy2013 @byandrewwagner @themikebpeters dr. cox, does this shade of red make me look like a clown?no barbie. it makes you look like a prostitute who caters exclusively to clowns.
2: sharon: oh, loki, does this lipstick make me look like a clown?loki: no, barbie, no... it makes you look like a prostitute who caters exclusively *to* clowns.
Cosine Similarity: 0.4364
--------------------------------
Vocabulary size for non sexist trigrams of words:  21607
(1253, 1263)
Most similar non sexist tweets:
1: @jennfrey must be based on the bem sex role inventory and
2: bem sex role inventory is a problematic tool
Cosine Similarity: 0.5000


### TF-IDF

In [111]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

counter = CountVectorizer(stop_words="english",binary=False)
tfidf = TfidfTransformer(norm="l2")

# Sexist tweets
dtMatrix_sexist = counter.fit_transform(MyCorpus_sexist).toarray()
tfidfMatrix_sexist = tfidf.fit_transform(dtMatrix_sexist).toarray()

print("TF-IDF sexist:", tfidfMatrix_sexist.shape)
compute_similarity(MyCorpus_sexist, tfidfMatrix_sexist, True)

print("--------------------------------")

# Non-sexist tweets
dtMatrix_nonsexist = counter.fit_transform(MyCorpus_nonsexist).toarray()
tfidfMatrix_nonsexist = tfidf.fit_transform(dtMatrix_nonsexist).toarray()

print("TF-IDF non-sexist:", tfidfMatrix_nonsexist.shape)
compute_similarity(MyCorpus_nonsexist, tfidfMatrix_nonsexist, False)

TF-IDF sexist: (1130, 6457)
(96, 628)
Most similar sexist tweets:
1: @davidwa41615213 blah blah…she writes as good as a man…blah blah
2: @olisaosega another one wey dey always vex me for this app na when person talk say "men do this..." "women do this....""blah blah blah does this..." someone will come it "some men, not all men" etc. like they are trying so desperately to not get (oppose) the message
Cosine Similarity: 0.5611
--------------------------------
TF-IDF non-sexist: (1722, 10329)
(635, 639)
Most similar non sexist tweets:
1: @bleedthisway replay free woman breebylon &gt;&gt;&gt; flop this way
2: replay&gt;alice&gt;babylon&gt;free woman 
Cosine Similarity: 0.7430


### LSA 50 components

In [112]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD

counter = CountVectorizer(stop_words="english",binary=False)
tfidf = TfidfTransformer(norm="l2")
svd = TruncatedSVD(n_components = 50)

# Sexist tweets
dtMatrix_sexist = counter.fit_transform(MyCorpus_sexist).toarray()
tfidfMatrix_sexist = tfidf.fit_transform(dtMatrix_sexist).toarray()
svdMatrix_sexist = svd.fit_transform(tfidfMatrix_sexist)

print("After LSA for sexist tweets (LSA for 50 representation):", svdMatrix_sexist.shape)
compute_similarity(MyCorpus_sexist, svdMatrix_sexist, True)

print("--------------------------------")

# Non-sexist tweets
dtMatrix_nonsexist = counter.fit_transform(MyCorpus_nonsexist).toarray()
tfidfMatrix_nonsexist = tfidf.fit_transform(dtMatrix_nonsexist).toarray()
svdMatrix_nonsexist = svd.fit_transform(tfidfMatrix_nonsexist)

print("After LSA for non-sexist tweets (LSA for 50 representation):", svdMatrix_nonsexist.shape)
compute_similarity(MyCorpus_nonsexist, svdMatrix_nonsexist, False)


After LSA for sexist tweets (LSA for 50 representation): (1130, 50)
(395, 398)
Most similar sexist tweets:
1: lollll fuck that cunt ahahhaha 
2: fuck that cunt get joffy on 
Cosine Similarity: 0.9999
--------------------------------
After LSA for non-sexist tweets (LSA for 50 representation): (1722, 50)
(255, 262)
Most similar non sexist tweets:
1: @sliturguts @inhumanew4ste cant boss hairs soakin
2: @manutd ralf rangnick 🙏 be our boss 
Cosine Similarity: 1.0000


### LSA 100 components

In [113]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD

counter = CountVectorizer(stop_words="english",binary=False)
tfidf = TfidfTransformer(norm="l2")
svd = TruncatedSVD(n_components = 100)

# Sexist tweets
dtMatrix_sexist = counter.fit_transform(MyCorpus_sexist).toarray()
tfidfMatrix_sexist = tfidf.fit_transform(dtMatrix_sexist).toarray()
svdMatrix_sexist = svd.fit_transform(tfidfMatrix_sexist)

print("After LSA for sexist tweets (LSA for 100 representation):", svdMatrix_sexist.shape)
compute_similarity(MyCorpus_sexist, svdMatrix_sexist, True, )

print("--------------------------------")

# Non-sexist tweets
dtMatrix_nonsexist = counter.fit_transform(MyCorpus_nonsexist).toarray()
tfidfMatrix_nonsexist = tfidf.fit_transform(dtMatrix_nonsexist).toarray()
svdMatrix_nonsexist = svd.fit_transform(tfidfMatrix_nonsexist)

print("After LSA for non-sexist tweets (LSA for 100 representation):", svdMatrix_nonsexist.shape)
compute_similarity(MyCorpus_nonsexist, svdMatrix_nonsexist, False)


After LSA for sexist tweets (LSA for 100 representation): (1130, 100)
(395, 398)
Most similar sexist tweets:
1: lollll fuck that cunt ahahhaha 
2: fuck that cunt get joffy on 
Cosine Similarity: 0.9996
--------------------------------
After LSA for non-sexist tweets (LSA for 100 representation): (1722, 100)
(255, 262)
Most similar non sexist tweets:
1: @sliturguts @inhumanew4ste cant boss hairs soakin
2: @manutd ralf rangnick 🙏 be our boss 
Cosine Similarity: 0.9998
